In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

## Deleting J02, J04, J05

In [3]:
# Importing
one_df_days = pd.read_pickle(r'C:\Users\a.bagherian\Desktop\working\one_df_days\one_df_days_18_95.pickle')

In [ ]:
one_df_days

In [7]:
# Importing service
service = pd.read_pickle(r'C:\Users\a.bagherian\Desktop\IHIO_share_A\service.pkl')
service.set_index('service', inplace=True)

In [11]:
atc_to_delete = ['J02', 'J04', 'J05']
service_to_delete = service.loc[service['atc_2'].isin(atc_to_delete)].index

In [15]:
one_df_days['service'].isin(service_to_delete).sum() / one_df_days.shape[0]

0.05079114484789126

In [19]:
one_df_days_J01 = one_df_days.loc[~ one_df_days['service'].isin(service_to_delete)].copy()

In [27]:
1 - (one_df_days_J01['days'] / pd.Timedelta(days=1)).sum() / (
    one_df_days['days'] / pd.Timedelta(days=1)).sum()

0.048579555731010804

In [29]:
# Saving one_df_days_J01
# one_df_days_J01.to_pickle(r'E:\after_session_03_22\data\one_df\one_df_days_18_95_J01.pkl')

## Building member_days_18_95_J01

In [ ]:
one_df_days_J01

### Removing outliers

In [ ]:
# Removing outliers
limit = pd.Timedelta(30, 'day')
print(limit)
outliers = one_df_days_J01.loc[one_df_days_J01['days'] > limit]
outliers

In [36]:
one_df_days_J01_outliers_removed = one_df_days_J01.loc[one_df_days_J01['days'] <= limit].copy()

In [38]:
one_df_days_J01_outliers_removed.describe()

,days
count,9017002
mean,7 days 17:12:35.776860208
std,5 days 07:33:58.336634953
min,0 days 00:01:09.120000
25%,4 days 23:59:42.720000
50%,6 days 15:59:02.400000
75%,10 days 00:00:00
max,30 days 00:00:00


In [39]:
member_days_J01 = one_df_days_J01_outliers_removed.groupby('member')['days'].sum()

In [41]:
member_days_J01.describe() / 3

count                 464165.666667
mean     16 days 15:46:14.416763567
std      20 days 16:14:32.299356431
min          0 days 00:01:35.040000
25%                 5 days 00:00:00
50%         10 days 13:19:35.040000
75%      20 days 17:19:11.999999999
max       1853 days 07:42:08.640000
Name: days, dtype: object

In [42]:
# Saving member_days_18_95_outliers_removed_J01
# member_days_J01.to_pickle(r'E:\after_session_03_22\data\one_df\member_days_18_95_outliers_removed_J01.pkl')

### Building member_cat_J01

In [43]:
days_final = pd.read_pickle(r'E:\after_session_03_22\data\one_df\member_days_18_95_outliers_removed_J01.pkl')

In [ ]:
days_final = days_final.to_frame()
days_final

In [54]:
days_final.loc[days_final['days'] > pd.Timedelta(0, 'day')].describe() / 3

,days,cat
count,464165.666667,464165.666667
mean,16 days 15:46:14.416763567,0.823999
std,20 days 16:14:32.299356431,0.373586
min,0 days 00:01:35.040000,0.333333
25%,5 days 00:00:00,0.333333
50%,10 days 13:19:35.040000,0.666667
75%,20 days 17:19:11.999999999,1.000000
max,1853 days 07:42:08.640000,1.333333


In [ ]:
num_days = (days_final['days'] / pd.Timedelta(1, 'day')) / 3
num_days 

In [49]:
days_final.loc[num_days == 0, 'cat'] = 0
days_final.loc[(0 < num_days) & (num_days <= 5), 'cat'] = 1
days_final.loc[(5 < num_days) & (num_days <= 11), 'cat'] = 2
days_final.loc[(11 < num_days) & (num_days <= 21), 'cat'] = 3
days_final.loc[(21 < num_days), 'cat'] = 4

In [ ]:
days_final

In [51]:
days_final['cat'].value_counts(normalize=True, dropna=False)

1.0    0.257897
2.0    0.257753
4.0    0.245543
3.0    0.238807
Name: cat, dtype: float64

In [55]:
# saving member_cat_J01
# days_final.to_pickle(r'E:\after_session_03_22\data\one_df\member_cat_J01.pkl')

## Building person_df_J01

In [ ]:
person_df = pd.read_pickle(r'E:\after_session_03_22\data\person_df_3agegroups.pkl')
person_df

In [58]:
person_df.drop('cat', axis=1, inplace=True)

In [59]:
# Importing member_cat_J01
member_cat_J01 = pd.read_pickle(r'E:\after_session_03_22\data\one_df\member_cat_J01.pkl')

In [ ]:
member_cat_J01

In [61]:
person_df_J01 = pd.merge(
    person_df,
    member_cat_J01[['cat']],
    how='left',
    left_index=True,
    right_index=True
)

In [63]:
person_df_J01['cat'].value_counts(normalize=True, dropna=False)

1.0    0.210729
2.0    0.210611
4.0    0.200635
3.0    0.195131
NaN    0.182894
Name: cat, dtype: float64

In [67]:
person_df_J01['cat'].fillna(0, inplace=True)

In [71]:
person_df_J01['cat'] = person_df_J01['cat'].astype(int)

In [ ]:
person_df_J01

In [74]:
# Saving person_df_J01
# person_df_J01.to_pickle(r'E:\after_session_03_22\data\one_df\person_df_J01.pkl')

## Building parts with new ab column just based on J01

In [76]:
ab_service = service.loc[service['atc_2'] == 'J01'].index

In [ ]:
ab_service

In [82]:
prev_parts_path = Path(r'C:\Users\a.bagherian\Desktop\working\Diabetics Combined 18_95')
os.listdir(prev_parts_path)

['part1_dm_combined_18_95.pkl',
 'part2_dm_combined_18_95.pkl',
 'part3_dm_combined_18_95.pkl',
 'part4_dm_combined_18_95.pkl',
 'part5_dm_combined_18_95.pkl',
 'part6_dm_combined_18_95.pkl',
 'part7_dm_combined_18_95.pkl',
 'part8_dm_combined_18_95.pkl']

In [122]:
# Applying new ab columns and saving

# for part_num, prev_loc in enumerate(os.listdir(prev_parts_path)):
#     prev_loc = prev_parts_path / prev_loc
#     part_num += 1

#     # Importing previous part
#     part = pd.read_pickle(prev_loc)

#     # Modifying ab column to only consider J01
#     part['ab'] = part['service'].isin(ab_service)

#     # Modifying days column to only consider J01
#     part.loc[~ part['ab'], 'days'] = pd.Timedelta(0, 'day')

#     # Merging data from person_df_J01
#     part = pd.merge(
#         part,
#         person_df_J01.loc[
#             person_df_J01['part'] == part_num, ['fund', 'age', 'age_cat', 'cat']],
#         how='left',
#         left_on='member',
#         right_index=True
#     )
    
#     # Saving new parts
#     new_part_loc = (
#         Path(r'E:\after_session_03_22\data\part_data') / (prev_loc.stem + '_J01' + prev_loc.suffix)
#     )
#     part.to_pickle(new_part_loc)

In [ ]:
part

## Replacing mod of province with province of parts

In [ ]:
# Importing person_df_J01
person_df_J01 = pd.read_pickle(r'E:\after_session_03_22\data\one_df\person_df_J01.pkl')
person_df_J01

In [4]:
parts_path = Path(r'E:\after_session_03_22\data\part_data')
os.listdir(parts_path)

['part1_dm_combined_18_95_J01.pkl',
 'part2_dm_combined_18_95_J01.pkl',
 'part3_dm_combined_18_95_J01.pkl',
 'part4_dm_combined_18_95_J01.pkl',
 'part5_dm_combined_18_95_J01.pkl',
 'part6_dm_combined_18_95_J01.pkl',
 'part7_dm_combined_18_95_J01.pkl',
 'part8_dm_combined_18_95_J01.pkl']

In [7]:
# Adding prov_mod column to parts

person_df_J01.rename(columns={'province': 'prov_mod'}, inplace=True)

for part_num, part_loc in enumerate(os.listdir(parts_path)):
    part_loc = parts_path / part_loc
    part_num += 1

    # Importing previous part
    part = pd.read_pickle(part_loc)

    # Adding prov_mod
    part = pd.merge(
        part,
        person_df_J01.loc[
            person_df_J01['part'] == part_num, ['prov_mod']
        ],
        how='left',
        left_on='member',
        right_index=True
    )

    # Saving new parts
    new_part_loc = (
        Path(r'E:\after_session_03_22\data\part_data\prov_mod') / part_loc.name
    )
    part.to_pickle(new_part_loc)